# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

## 1.1 Installation

In [1]:
!pip install -q datasets boto3

## 1.2 Imports

In [2]:
# Import all Python packages required to access the Bedrock Claude API.
# Import additional packages required to access datasets and create examples.

import boto3
import json
import random

import pandas as pd
import numpy as np

from datasets import load_dataset
from collections import Counter
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tabulate import tabulate

## 1.3 Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [3]:
!mkdir ~/.aws
!cp aws_config.ini ~/.aws/credentials

In [4]:
client = boto3.client('bedrock-runtime')

In [5]:
model_id = 'anthropic.claude-instant-v1'

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

For the sentiment analysis classification task, we will use a dataset of courier service reviews for ExpressWay Logistics. Our investigation will focus on assigning positive or negative sentiment to courier service reviews that customers have posted on online channels. During prompt engineering, we will use a hold-out set of reviews (i.e., gold examples) to ascertain the quality of the sentiment assignment.

**(A) Upload and read csv file (2 Marks)**

In [6]:
cs_reviews_df = pd.read_csv('courier-service_reviews.csv')
# Read CSV File Here

In [7]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [8]:
cs_reviews_df.sample(5)

,id,review,sentiment
24,25,ExpressWay Logistics helped me navigate the cu...,Positive
79,80,ExpressWay Logistics' lack of attention to det...,Negative
90,91,ExpressWay Logistics' proactive approach to pr...,Positive
9,10,My interactions with ExpressWay Logistics have...,Negative
77,78,ExpressWay Logistics is unreliable and untrust...,Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

2. Create a new column named as "label" (target column) corresponding to the sentiments in the dataset.

In [9]:
cs_reviews_df['label'] = np.where(cs_reviews_df.sentiment == "Positive", 1,0)

In [10]:
cs_reviews_df.sample(5)

,id,review,sentiment,label
58,59,I am extremely dissatisfied with the service p...,Negative,0
92,93,The real-time notifications and updates provid...,Positive,1
106,107,ExpressWay Logistics makes shipping hassle-fre...,Positive,1
31,32,The packaging provided by ExpressWay Logistics...,Positive,1
128,129,I had a question about shipping regulations fo...,Positive,1


In [11]:
cs_reviews_df.shape

(131, 4)

In [12]:
cs_reviews_df['sentiment'].value_counts()

,count
sentiment,
Positive,68
Negative,63


In [13]:
cs_reviews_df['label'].value_counts()

,count
label,
1,68
0,63


**There are 68 reviews labeled with positive sentiment, and 63 reviews labeled with negative sentiment.**

**(C) Split the Dataset (2 Marks)**

3. Now that the preprocessing is done, let us split the data into two segments (use split_ratio of 0.2) - one segment (80%) that gives us a pool to draw few-shot examples from and another segment (20%) that gives us a pool of gold examples.

In [14]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

In [15]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 4), (27, 4))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

Select the correct columns for further analysis which should exclude the target column.

In [16]:
columns_to_select = ['review','sentiment']

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [17]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

Print the gold_examples

In [18]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [19]:
json.loads(gold_examples)[0]     #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

### Step 3: Derive Prompt

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

#### Create prompts

In [20]:
claude_prompt_template = """

Human: {system_message}
{prompt}

Assistant: {response}
"""

In [21]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [22]:
zero_shot_system_message = """Please classify the sentiment of the customer reviews in the input as either Positive or Negative.
The review will be delimited by triple backticks, that is, ```.
I do not want any details or elaboration. I simply want the sentiment review to be classified as either: Positive or Negative.
"""
# Write zero shot system message here

**(B) Create Zero Shot Prompt (2 Marks)**

In [23]:
zero_shot_prompt = f"""Human: {zero_shot_system_message}

Human: {user_message_template}

Assistant: """
# Create zero shot prompt to be input ready for completion function

**(C) Write Few Shot System Message (3 Marks)**

**Prompt 2: Few-shot**

For the few-shot prompt, there is no change in the system message compared with the zero-shot prompt. However, we augment this system message with few shot examples.  

In [24]:
few_shot_system_message = """Please classify the sentiment of the customer reviews in the input as either Positive or Negative.
The review will be delimited by triple backticks, that is, ```.
I do not want any details or elaboration. I simply want the sentiment review to be classified as either: Positive or Negative.
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [25]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**(D) Create Examples For Few shot prompte (2 Marks)**

In [26]:
examples = create_examples(cs_examples_df)
# Create Examples

In [27]:
json.loads(examples)

[{'review': "ExpressWay Logistics' comprehensive tracking system provides real-time updates on the status of our shipments, giving us peace of mind throughout the shipping process. Their advanced tracking capabilities allow us to monitor shipments closely and proactively address any issues that may arise. With ExpressWay Logistics, we can trust that our packages are in good hands.",
  'sentiment': 'Positive'},
 {'review': 'I appreciate the attention to detail that ExpressWay Logistics puts into every aspect of their service, from packaging to delivery to customer support and the promised door to door deliveries.Overall a satisfied service!',
  'sentiment': 'Positive'},
 {'review': "I encountered numerous issues with ExpressWay Logistics' customer service department when trying to resolve a problem with my shipment. Despite multiple attempts to reach them by phone and email, I was met with long wait times and unhelpful responses from representatives who seemed uninformed about company p

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [28]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = ''

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']
        few_shot_prompt += claude_prompt_template.format(
                system_message=system_message,
                prompt=user_message_template.format(courier_service_review=example_review),
                response=example_sentiment
        )

    return (
        few_shot_prompt +
        claude_prompt_template.format(
            system_message=system_message,
            prompt=user_message_template,
            response=''
        )
    )

**(E) Create Few Shot Prompt (2 Marks)**

In [29]:
few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)
# Create Few shot prompt

In [30]:
few_shot_prompt

"\n\nHuman: Please classify the sentiment of the customer reviews in the input as either Positive or Negative.\nThe review will be delimited by triple backticks, that is, ```.\nI do not want any details or elaboration. I simply want the sentiment review to be classified as either: Positive or Negative.\n\n```ExpressWay Logistics' comprehensive tracking system provides real-time updates on the status of our shipments, giving us peace of mind throughout the shipping process. Their advanced tracking capabilities allow us to monitor shipments closely and proactively address any issues that may arise. With ExpressWay Logistics, we can trust that our packages are in good hands.```\n\nAssistant: Positive\n\n\nHuman: Please classify the sentiment of the customer reviews in the input as either Positive or Negative.\nThe review will be delimited by triple backticks, that is, ```.\nI do not want any details or elaboration. I simply want the sentiment review to be classified as either: Positive or

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [31]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']

        try:
            payload = json.dumps({
                "prompt": prompt.format(courier_service_review=gold_input),
                "max_tokens_to_sample": 2,
                "temperature": 0
            })

            response = client.invoke_model(
                body=payload,
                modelId=model_id
            )

            response_body = json.loads(response['body'].read())

            prediction = response_body['completion']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [32]:
zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

In [33]:
print(f"The Micro-F1 score for Zero Shot prompts is {zero_shot_micro_f1}.")

The Micro-F1 score for Zero Shot prompts is 0.8571428571428571.


**(B) Evaluate few shot prompt (2 Marks)**

In [34]:
few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

In [35]:
print(f"The Micro-F1 score for Few Shot prompts is {few_shot_micro_f1}.")

The Micro-F1 score for Few Shot prompts is 0.9523809523809523.


However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [36]:
num_eval_runs = 5

In [37]:
zero_shot_performance = []
few_shot_performance = []

In [38]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt
    zero_shot_prompt = f"""Human: {zero_shot_system_message}\n\nHuman: {user_message_template}\n\nAssistant: """

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [00:16<01:04, 16.07s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [00:34<00:52, 17.38s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [01:04<00:15, 15.93s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [01:19<00:00, 15.81s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [39]:
print(f"""Based on the above evaluation, the Zero Shot prompt has an average score of {np.mean(zero_shot_performance)},
with a standard deviation of {np.std(zero_shot_performance)}.""")
# Calculate for Zero Shot

Based on the above evaluation, the Zero Shot prompt has an average score of 0.8571428571428571, 
with a standard deviation of 0.0.


In [40]:
print(f"""Based on the above evaluation, the Few Shot prompt has an average score of {np.mean(few_shot_performance)},
with a standard deviation of {np.std(few_shot_performance)}.""")
# Calculate for Few Shot

Based on the above evaluation, the Few Shot prompt has an average score of 0.980952380952381, 
with a standard deviation of 0.023328473740792197.


##**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified. )


# **RESPONSE:**

Based on this analysis of the customer feedback for ExpressWay Logistics in the ``courier-service_reviews.csv`` document, it is clear that barely half of the customers (52%) in the analyzed sample of reviews appeared to give positive feedback, while the remaining customers (48%) had negative feedback. This indicates that a great deal of improvement can be done to achieve higher levels of customer satisfaction.

To facilitate the process of analyzing future customer sentiment for this business, we deployed a natural language AI model to read the customer reviews from that CSV file, and judge whether they had Positive or Negative feedback, independent of what the ``sentiment`` column indicated. We then compared the model's classification of those reviews to the original classification from the ``sentiment`` column.

We tested two different versions of models. The first version used a very simple Zero Shot Prompt, in which the model judged the sentiment of the reviews based on nothing more than its baseline training. However, the second version used what we call a Few Shot Prompt, in which we fed a few examples of what positive and negative reviews look like, in order to give some additional training to the model to better recognize the sentiment.

**Our analysis shows that the model version with the Few Shot Prompt has the highest accuracy, with an average accuracy score of 98%, compared to the Zero Shot Prompt at 86%. Furthermore, the Few Shot Prompt had very little deviation in its exceptional accuracy.**

**Therefore, we recommend the model that uses the Few Shot Prompt, because it will be a highly efficient and accurate way to automatically assess the sentiment of future customer feedback. Our hope is that improvements in customer service and transportation logistics will result in a greater proportion of reviews that show positive feedback, with a future of improved customer satisfaction.**